# Libxnd with pyobjects

In this notebook I will go over ways in which using the pure C apis (`libxnd`, `libndtypes`) on `XndObject`s from Python fail currently and how the exposed Python APIs help.  

First, let's start with going over how we get a `xnd_t` from a passed in xnd object:

In [1]:
import numba_xnd
from numba import njit
from xnd import xnd


In [2]:
@njit
def get_xnd_t(x_object_wrapper):
    # this is now an `XndObject`, as declared in `pyxnd.py`.
    # unwrapping it allows us to directly use it as a C struct instead of 
    # as a python object.
    x_object = numba_xnd.pyxnd.unwrap_xnd_object(x_object_wrapper)
    # we can get a pointer to the `xnd_t` struct from it
    # this type is declared in `libxnd.py`
    x = x_object.xnd
    # We canreturn the address of the pointer of the data
    return numba_xnd.shared.ptr_to_int(x.ptr)
    
get_xnd_t(xnd(10))

140222450068736

Now let's do some slicing. Ideally, we want to be able to just deal with `xnd_t` object and only convert to `XndObject`s during boxing. So let's try to use the `xnd_subtree` function to do some indexing and then box the result with `xnd_from_type_xnd`. This works for simple types:

In [3]:
@njit
def try_slicing(x_object_wrapper, i):
    x_object = numba_xnd.pyxnd.unwrap_xnd_object(x_object_wrapper)
    x = x_object.xnd
    
    index = numba_xnd.libxnd.create_xnd_index()
    index.tag = numba_xnd.libxnd.XND_KEY_INDEX
    index.Index = i
    ret_xnd = numba_xnd.libxnd.create_xnd()
    ctx = numba_xnd.libndtypes.ndt_static_context()
    numba_xnd.libxnd.xnd_subtree(
        ret_xnd, x, index, numba_xnd.shared.i64_to_i32(1), ctx
    )
    assert not numba_xnd.shared.ptr_is_none(ret_xnd.ptr)
    assert not numba_xnd.libndtypes.ndt_err_occurred(ctx)
    
    return numba_xnd.pyxnd.xnd_from_type_xnd(
        numba_xnd.pyxnd.get_xnd_type_obj(),
        ret_xnd
    )

In [4]:
x = xnd([1, 2, 3, 4])

In [5]:
try_slicing(x, 3)

xnd(4, type='int64')

In [6]:
try_slicing(x, 1)

xnd(2, type='int64')

In [7]:
try_slicing(xnd([[1, 2], [3, 4]]), 1)

xnd([3, 4], type='2 * int64')

In [8]:
x

xnd([1, 2, 3, 4], type='4 * int64')

OK That seems to work pretty well. Let's try this with the high level API:

In [9]:
@njit
def slicing_high_level(x_object_wrapper, i):
    x_object = numba_xnd.pyxnd.unwrap_xnd_object(x_object_wrapper)
    x = x_object.xnd
    x_wrapped = numba_xnd.libxnd.wrap_xnd(x, x_object_wrapper)
    ret_x_wrapped = x_wrapped[i]
    ret_x = numba_xnd.libxnd.unwrap_xnd(ret_x_wrapped)
    return numba_xnd.pyxnd.xnd_from_type_xnd(
        numba_xnd.pyxnd.get_xnd_type_obj(),
        ret_x
    )

In [ ]:
slicing_high_level(x, 1)